# Intelligent Crowd Analytics for Robust and Ubiquitous Surveillance - ICARUS
### Intelligent Sensing Systems Practice Module 2022

- Analytics engine capable of providing persistent monitoring on both indoor and outdoor static CCTVs to provide higher-order insights on crowd social behaviour
- Modules include:
    - Social Distancing Monitoring (COVID-19)
    - Crowd Management (Crowd Counting)
    - Crowd Anomaly (Flow Estimation/Stampede Alert)
    - Fall Detection (Medical Emergencies)
    - Violent Behaviour Detection (Fighting, assualt)

Team: Adriel Kuek, Chua Hao Zi, KC lim & Yap Pow Look

In [ ]:
# Import
import torch
from IPython.display import Image, clear_output

clear_output()
print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

# limit the number of cpus used by high performance libraries
import os
os.environ["OMP_NUM_THREADS"] = "1"
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["MKL_NUM_THREADS"] = "1"
os.environ["VECLIB_MAXIMUM_THREADS"] = "1"
os.environ["NUMEXPR_NUM_THREADS"] = "1"

import sys
sys.path.insert(0, './yolov5')

import argparse
import os
import platform
import shutil
import time
from pathlib import Path
import cv2
import torch
import torch.backends.cudnn as cudnn

from yolov5.models.experimental import attempt_load
from yolov5.utils.downloads import attempt_download
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.datasets import LoadImages, LoadStreams
from yolov5.utils.general import (LOGGER, check_img_size, non_max_suppression, scale_coords, 
                                  check_imshow, xyxy2xywh, increment_path)
from yolov5.utils.torch_utils import select_device, time_sync
from yolov5.utils.plots import Annotator, colors
from deep_sort.utils.parser import get_config
from deep_sort.deep_sort import DeepSort

# FILE = Path(__file__).resolve()
ROOT = os.path.abspath('')
if str(ROOT) not in sys.path:
    sys.path.append(str(ROOT))  # add ROOT to PATH
ROOT = Path(os.path.relpath(ROOT, Path.cwd()))  # relative

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

print(sys.path)
print(f'Compute Device: {device}')


# Initialise Models
## YOLOv5 + DeepSORT

In [ ]:
evaluate = True
half = True
output = 'output/'
imgsz = [640, 640]
device = select_device(device)

# Initialise DeepSORT
cfg = get_config()
cfg.merge_from_file('deep_sort/configs/deep_sort.yaml')
deepsort = DeepSort('osnet_x0_25',
                    device,
                    max_dist=cfg.DEEPSORT.MAX_DIST,
                    max_iou_distance=cfg.DEEPSORT.MAX_IOU_DISTANCE,
                    max_age=cfg.DEEPSORT.MAX_AGE,
                    n_init=cfg.DEEPSORT.N_INIT,
                    nn_budget=cfg.DEEPSORT.NN_BUDGET)

# Initialise half precision - Only for CUDA enabled devices
half &= device.type != 'cpu'

# For MOT16 evaluation - Run multiple inference streams in parallel
if not evaluate:
    if os.path.exists(output):
        pass
        shutil.rmtree(output)
    os.makedirs(output)

# Directories
save_dir = increment_path(Path(ROOT) / 'exp', exist_ok=True)
save_dir.mkdir(parents=True, exist_ok=True)  # make dir

# Load YOLO Model
model_yolo = DetectMultiBackend('yolov5/models/crowdhuman_yolov5m.pt',
                            device=device,
                            dnn=True)
stride, names, pt, jit, _ = model_yolo.stride, model_yolo.names, model_yolo.pt, model_yolo.jit, model_yolo.onnx
imgsz = check_img_size(imgsz, s=stride)

half &= pt and device.type != 'cpu'
if pt:
    model_yolo.model.half() if half else model_yolo.model.float()

# Load Functions for Violence Detection
### Framework uses C3D with 5 secs worth of video stream

In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Activation, Conv3D, Dense, Dropout, Flatten, MaxPooling3D
from tensorflow.keras.losses import categorical_crossentropy, binary_crossentropy
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.models import load_model

# Default Parameters
c3d_width = 32
c3d_height = 32
c3d_depth = 10

class Datato3D:

    def __init__(self, depth, c3d_model):

        self.depth = depth
        self.c3d_model = c3d_model

    def videoto3D(self, videoarray):
        '''
        input : video numpy data in [nframes, 32,32]
        output: vid3D file in [10,32,32] format
        '''

        vid3D = []
        nframes = videoarray.shape[0]

        if (nframes >= self.depth):
            frames = [x * nframes / self.depth for x in range(self.depth)]
        else:
            frames = [x for x in range(int(nframes))]

        for i in range(len(frames)):
            vid3D.append(videoarray[int(frames[i]), :, :])

        return np.array(vid3D)

    def inference(self, video_frames):

        category = {0: "No Violence",
                    1: "Violence"}

        # c3D_model = load_model(os.path.join(os.getcwd(), self.model_path))
        # c3D_model.load_weights(os.path.join(os.getcwd(), self.weights_path))
        vid3D = self.videoto3D(video_frames)
        vid3D = np.expand_dims(vid3D, axis=3)
        vid3D = vid3D.transpose((1, 2, 0, 3))
        vid3D = np.expand_dims(vid3D, axis=0)

        y_pred = self.c3d_model.predict(vid3D)

        # return category[np.argmax(y_pred)]
        return np.argmax(y_pred)

## Initialise C3D model

In [ ]:
c3d_model_path = 'ViolenceDetection_Models/c3d_model.h5'
c3d_model_weights = 'ViolenceDetection_Models/c3d_model_120f_weights.h5'

c3d_model = load_model(c3d_model_path)
c3d_model.load_weights(c3d_model_weights)

datato3D = Datato3D(c3d_depth, c3d_model)

# Video Extraction Module
## Extract video segment
## Convert video to image frames

In [ ]:
show_vid = True

# Video Source
source = 'SampleVideos/TownCentre_short.mp4'

# Set Dataloader
vid_path, vid_writer = None, None

# Check if environment supports image displays
if show_vid:
    show_vid = check_imshow()
    print(f'show_vid: {show_vid}')

# Dataloader
dataset = LoadImages(source, img_size=imgsz, stride=stride, auto=pt and not jit)
bs = 1  # batch_size
vid_path, vid_writer = [None] * bs, [None] * bs

In [ ]:
# Get Names and Colours
names = model_yolo.module.names if hasattr(model_yolo, 'module') else model_yolo.names

# extract filename
txt_file_name = source.split('/')[-1].split('.')[0]
txt_path = str(Path(save_dir)) + '/' + txt_file_name + '.txt'

if pt and device.type != 'cpu':
    model_yolo(torch.zeros(1, 3, *imgsz).to(device).type_as(next(model_yolo.model.parameters())))  # warmup
dt, seen = [0.0, 0.0, 0.0, 0.0], 0

In [ ]:
if txt_file_name == 'TownCentre_short':
    bool_towncenter = True
else:
    bool_towncenter = False

if txt_file_name == 'AppleStoreShooting_short1':
    bool_applestore = True
else:
    bool_applestore = False

# Configure Detector parameters

In [ ]:
# Configuration
augment = True          # Augmented inference
visual = False          # Visualisation function - Set this to false as default
conf_thres = 0.3        # Object confidence threshold
iou_thres = 0.5         # IOU Threshold for NMS
classes = 0             # Filter for class 0 - Person
agnostic_nms = True     # Class agnostic NMS
max_det = 1000          # Max number of detections per image
save_txt = False
save_vid = True

# Social Distance Metrics
## SD Violations version 1 - using tracking box centroid framework.
## Initialise hyperparameters for social distance model

In [ ]:
# Skip this cell
%script false

import socialdistance as sd

# Configuration
dist_thres = 1.0
violation_thres = 5
DepthControlFactor = 0.3
height_control = 1.7
cam = 1

# Global Dictionaries
socialdistance_dict = {}
printed_tracks = []
track_pair_list = []
count = 0
# filter inaccurate yolo bboxs
seen_tracks = {}

# Initialise Model with input configuration
sd = sd.socialdistance(dist_thres, violation_thres, DepthControlFactor, height_control, socialdistance_dict, seen_tracks, count, track_pair_list, printed_tracks)


# Iterate across images stored in pytorch dataloader and perform detection and tracking
# - Compute Social Distance Violation

In [ ]:
# Skip this cell
%script false

with torch.no_grad():
    for frame_idx, (path, img, im0s, vid_cap, s) in enumerate(dataset):

        # Initialise every frame for socialdistance model
        img_centroid = []
        trackid_bbox_centroid = []

        t1 = time_sync()
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visual else False
        pred = model_yolo(img, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            seen += 1
            p, im0, _ = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name) # im.jpg, vid.mp4, ...
            s += '%gx%g ' % img.shape[2:]  # print string

            # annotator = Annotator(im0, line_width=2, pil=not ascii)

            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(
                    img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                xywhs = xyxy2xywh(det[:, 0:4])
                confs = det[:, 4]
                clss = det[:, 5]

                # pass detections to deepsort
                t4 = time_sync()
                outputs = deepsort.update(xywhs.cpu(), confs.cpu(), clss.cpu(), im0)
                t5 = time_sync()
                dt[3] += t5 - t4

                # draw boxes for visualization
                if len(outputs) > 0:
                    for j, (output, conf) in enumerate(zip(outputs, confs)):

                        bboxes = output[0:4]
                        id = output[4]
                        cls = output[5]

                        # Compute box centroid - Social Distance
                        img_centroid = [int(output[0] + (output[2] - output[0])/2), int(output[1]+ (output[3] - output[1])/2)]
                        img_height = output[3] - output[1]

                        # Box definitions
                        x = int(output[0])
                        y = int(output[1])
                        w = int(output[2] - output[0])
                        h = int(output[3] - output[1])
                        bbox_int = x,y,w,h

                        trackid_bbox_centroid.append(['{}'.format(id), '{}'.format(cam), bbox_int, img_centroid, img_height])

                        c = int(cls)  # integer class
                        label = f'ID:{id} ({conf:.2f})'

                        # Comment out this portion and draw our own boxes
                        # annotator.box_label(bboxes, label, color=colors(c, True))

                        if save_txt:
                            # to MOT format
                            bbox_left = output[0]
                            bbox_top = output[1]
                            bbox_w = output[2] - output[0]
                            bbox_h = output[3] - output[1]
                            # Write MOT compliant results to file
                            with open(txt_path, 'a') as f:
                                f.write(('%g ' * 10 + '\n') % (frame_idx + 1, id, bbox_left,  # MOT format
                                                                bbox_top, bbox_w, bbox_h, -1, -1, -1, -1))

                LOGGER.info(f'{s}Done. YOLO:({t3 - t2:.3f}s), DeepSort:({t5 - t4:.3f}s)')

            else:
                deepsort.increment_ages()
                LOGGER.info('No detections')

            # # Stream results
            # im0 = annotator.result()
            # if show_vid:
            #     cv2.imshow(str(p), im0)
            #     if cv2.waitKey(1) == ord('q'):  # q to quit
            #         raise StopIteration

            # Save results (image with detections)
            # if save_vid:
            #     if vid_path != save_path:  # new video
            #         vid_path = save_path
            #         if isinstance(vid_writer, cv2.VideoWriter):
            #             vid_writer.release()  # release previous video writer
            #         if vid_cap:  # video
            #             fps = vid_cap.get(cv2.CAP_PROP_FPS)
            #             w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
            #             h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
            #         else:  # stream
            #             fps, w, h = 30, im0.shape[1], im0.shape[0]

            #         vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
            #     vid_writer.write(im0)

        img_out = sd.proximity_evaluation(trackid_bbox_centroid, im0, cam)

        # Plot results
        cv2.imshow(str(p), img_out)
        if cv2.waitKey(1) == ord('q'):  # q to quit
            raise StopIteration

        # Save Video
        if save_vid:
            if vid_path != save_path:  # new video
                vid_path = save_path
                if isinstance(vid_writer, cv2.VideoWriter):
                    vid_writer.release()  # release previous video writer
                if vid_cap:  # video
                    fps = vid_cap.get(cv2.CAP_PROP_FPS)
                    w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                else:  # stream
                    fps, w, h = 30, img_out.shape[1], img_out.shape[0]

                vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h))
            vid_writer.write(img_out)
    
    vid_cap.release()
    vid_writer.release()
    cv2.destroyAllWindows()

# Automated Calibration-free BEV Projection
- Manual Labour
- Automated Bird's eye view through CNN
- Intrinsic camera parameters from Camcalib (SPEC)

In [ ]:
import os
import sys
import torch
import joblib
import argparse
import numpy as np
from tqdm import tqdm
from loguru import logger
from skimage.io import imsave
import matplotlib.pyplot as plt
%matplotlib inline
import cv2

from scipy.special import softmax

sys.path.append('')
from camcalib.vis_utils import show_horizon_line
from camcalib.model import CameraRegressorNetwork
from camcalib.pano_dataset import CameraRegressorDataset, ImageFolder
from camcalib.cam_utils import bins2vfov, bins2pitch, bins2roll, convert_preds_to_angles

from pare.utils.image_utils import denormalize_images
from pare.utils.train_utils import load_pretrained_model

CKPT = 'spec_data/camcalib/checkpoints/camcalib_sa_biased_l2.ckpt'

screenshot_dir = 'screenshots/'
if not os.path.exists(screenshot_dir):
    os.mkdir(screenshot_dir)

# Extract 1st frame from video and save it to perform camera parameters prediction
vidcap = cv2.VideoCapture(source)
success,image = vidcap.read()

# Save as jpg
filename_base = os.path.basename(source)
filename = os.path.splitext(filename_base)[0] + '.jpg'

if success:
    cv2.imwrite(os.path.join(screenshot_dir,filename), image)

In [ ]:
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

class ImageFolder(Dataset):
    def __init__(
            self,
            image_list,
            min_size=600,
            max_size=1000,
    ):
        self.image_filenames = image_list

        self.data_transform = transforms.Compose([
            transforms.Resize(min_size),
            # transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])
        ])

    def __len__(self):
        return len(self.image_filenames)

    def __getitem__(self, index):
        item = {}

        imgname = os.path.join(self.image_filenames[index])

        pil_img = Image.open(imgname).convert('RGB')
        orig_img_shape = pil_img.size
        norm_img = self.data_transform(pil_img)

        item['img'] = norm_img
        item['imgname'] = imgname

        item['orig_shape'] = orig_img_shape

        return item

## Prediction of camera intrinsic and extrinsic parameters
-vfov, focal length, tilt and roll

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt

loss_type = 'softargmax_l2'

with torch.no_grad():

    # Create dataloader list
    dataloader = []

    # Load image into dataloader
    image_list = [str(os.path.join(screenshot_dir,filename))]

    val_dataset = ImageFolder(image_list)

    model = CameraRegressorNetwork(
        backbone='resnet50',
        num_fc_layers=1,
        num_fc_channels=1024,
    ).to(device)

    camcalib_ckpt = torch.load(CKPT)
    camcalib_model = load_pretrained_model(model, camcalib_ckpt['state_dict'], remove_lightning=True, strict=True)

    logger.info('Loaded pretrained model')
    model.eval()

    focal_length = []

    logger.info('Running CamCalib')

    # Extract camera parameters across batch images (if multiple images)
    for idx, batch in enumerate(tqdm(val_dataset)):

        img_fname = batch['imgname']

        image_spec = batch['img'].unsqueeze(0).to(device).float()

        preds = camcalib_model(image_spec)
        preds_distributions = preds

        batch_img = image_spec
        batch_img = denormalize_images(batch_img) * 255
        batch_img = np.transpose(batch_img.cpu().numpy(), (0,2,3,1))

        extract = lambda x:x.detach().cpu().numpy().squeeze()
        img = batch_img[0].copy()

        if loss_type in ('kl', 'ce'):
            pred_vfov, pred_pitch, pred_roll = map(extract, preds)
            pred_vfov, pred_pitch, pred_roll = convert_preds_to_angles(
                                                pred_vfov, pred_pitch, pred_roll, loss_type=loss_type,
                                                return_type='np',)
        else:
            preds = convert_preds_to_angles(*preds, loss_type=loss_type,)
            pred_vfov = extract(preds[0])
            pred_pitch = extract(preds[1])
            pred_roll = extract(preds[2])
        
        orig_img_w, orig_img_h = batch['orig_shape']

        pred_f_pix = orig_img_h / 2. / np.tan(pred_vfov/2.)

        pitch = np.degrees(pred_pitch)
        roll = np.degrees(pred_roll)
        vfov = np.degrees(pred_vfov)

        # Save results into dictionary
        results = {}
        results['vfov'] = float(pred_vfov)
        results['f_pix'] = float(pred_f_pix)
        results['pitch'] = float(pred_pitch)
        results['roll'] = float(pred_roll)

        print("VFOV: {} rad".format(results['vfov']))
        print("Focal Length: {} px".format(results['f_pix']))
        print("Cam Pitch: {} rad".format(results['pitch']))
        print("Cam Roll: {} rad".format(results['roll']))
       
        focal_length.append(pred_f_pix)

        # Display horizontal line
        img, _ = show_horizon_line(img.copy(), pred_vfov, pred_pitch, pred_roll, focal_length=pred_f_pix,
                                   debug=True, color=(255, 0, 0), width=3, GT=False)
        
        plt.figure(figsize=[15,15])
        plt.imshow(img)
        plt.show()


## Geometric Projection Functions

In [ ]:
from utils_geometry.transformations import rotation_matrix

def modified_matrices_calculate_range_output_without_translation(height, width, overhead_hmatrix,
                                                                 verbose=False):
    range_u = np.array([np.inf, -np.inf])
    range_v = np.array([np.inf, -np.inf])

    i = 0
    j = 0
    u, v, w = np.dot(overhead_hmatrix, [j, i, 1])
    u = u / w
    v = v / w
    out_upperpixel = v
    if verbose:
        print(u, v)
    range_u[0] = min(u, range_u[0])
    range_v[0] = min(v, range_v[0])
    range_u[1] = max(u, range_u[1])
    range_v[1] = max(v, range_v[1])
    i = height - 1
    j = 0
    u, v, w = np.dot(overhead_hmatrix, [j, i, 1])
    u = u / w
    v = v / w
    out_lowerpixel = v
    if verbose:
        print(u, v)
    range_u[0] = min(u, range_u[0])
    range_v[0] = min(v, range_v[0])
    range_u[1] = max(u, range_u[1])
    range_v[1] = max(v, range_v[1])
    i = 0
    j = width - 1
    u, v, w = np.dot(overhead_hmatrix, [j, i, 1])
    u = u / w
    v = v / w
    if verbose:
        print(u, v)
    range_u[0] = min(u, range_u[0])
    range_v[0] = min(v, range_v[0])
    range_u[1] = max(u, range_u[1])
    range_v[1] = max(v, range_v[1])
    i = height - 1
    j = width - 1
    u, v, w = np.dot(overhead_hmatrix, [j, i, 1])
    u = u / w
    v = v / w
    if verbose:
        print(u, v)
    range_u[0] = min(u, range_u[0])
    range_v[0] = min(v, range_v[0])
    range_u[1] = max(u, range_u[1])
    range_v[1] = max(v, range_v[1])

    range_u = np.array(range_u, dtype=np.int)
    range_v = np.array(range_v, dtype=np.int)

    # it means that while transforming, after some bottom lower image was transformed,
    # upper output pixels got greater than lower
    if out_upperpixel > out_lowerpixel:

        # range_v needs to be updated
        max_height = height * 3
        upper_range = out_lowerpixel
        best_lower = upper_range  # since out_lowerpixel was lower value than out_upperpixel
        #                           i.e. above in image than out_lowerpixel
        x_best_lower = np.inf
        x_best_upper = -np.inf

        for steps_h in range(2, height):
            temp = np.dot(overhead_hmatrix, np.vstack(
                (np.arange(0, width), np.ones((1, width)) * (height - steps_h), np.ones((1, width)))))
            temp = temp / temp[2, :]

            lower_range = temp.min(axis=1)[1]
            x_lower_range = temp.min(axis=1)[0]
            x_upper_range = temp.max(axis=1)[0]
            if x_lower_range < x_best_lower:
                x_best_lower = x_lower_range
            if x_upper_range > x_best_upper:
                x_best_upper = x_upper_range

            if (upper_range - lower_range) > max_height:  # enforcing max_height of destination image
                lower_range = upper_range - max_height
                break
            if lower_range > upper_range:
                lower_range = best_lower
                break
            if lower_range < best_lower:
                best_lower = lower_range
            if verbose:
                print(steps_h, lower_range, x_best_lower, x_best_upper)
        range_v = np.array([lower_range, upper_range], dtype=np.int)

        # for testing
        range_u = np.array([x_best_lower, x_best_upper], dtype=np.int)

    return range_u, range_v
    
def get_overhead_hmatrix_from_4cameraparams(fx, fy, my_tilt, my_roll, img_dims, verbose=False):
    width, height = img_dims

    origin, xaxis, yaxis, zaxis = [0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]
    K3x3 = np.array([[fx, 0, width / 2],
                     [0, fy, height / 2],
                     [0, 0, 1]])

    inv_K3x3 = np.linalg.inv(K3x3)
    if verbose:
        print("K3x3:\n", K3x3)

    R_overhead = np.array([[1, 0, 0], [0, -1, 0], [0, 0, -1]])
    if verbose:
        print("R_overhead:\n", R_overhead)

    R_slant = rotation_matrix((pi / 2) + my_tilt, xaxis)[:3, :3]
    if verbose:
        print("R_slant:\n", R_slant)

    R_roll = rotation_matrix(my_roll, zaxis)[:3, :3]

    middle_rotation = np.dot(R_overhead, np.dot(np.linalg.inv(R_slant), R_roll))

    overhead_hmatrix = np.dot(K3x3, np.dot(middle_rotation, inv_K3x3))
    est_range_u, est_range_v = modified_matrices_calculate_range_output_without_translation(height, width,
                                                                                            overhead_hmatrix,
                                                                                            verbose=False)

    if verbose:
        print("Estimated destination range: u=", est_range_u, "v=", est_range_v)

    moveup_camera = np.array([[1, 0, -est_range_u[0]], [0, 1, -est_range_v[0]], [0, 0, 1]])
    if verbose:
        print("moveup_camera:\n", moveup_camera)

    overhead_hmatrix = np.dot(moveup_camera, np.dot(K3x3, np.dot(middle_rotation, inv_K3x3)))
    if verbose:
        print("overhead_hmatrix:\n", overhead_hmatrix)

    return overhead_hmatrix, est_range_u, est_range_v
    
def get_scaled_homography(H, target_height, estimated_xrange, estimated_yrange):
    # if don't want to scale image, then pass target_height = np.inf

    current_height = estimated_yrange[1] - estimated_yrange[0]
    target_height = min(target_height, current_height)
    (tw, th) = int(np.round((estimated_xrange[1] - estimated_xrange[0]))), int(
        np.round((estimated_yrange[1] - estimated_yrange[0])))

    tr = target_height / float(th)
    target_dim = (int(tw * tr), target_height)

    scaling_matrix = np.array([[tr, 0, 0], [0, tr, 0], [0, 0, 1]])
    scaled_H = np.dot(scaling_matrix, H)

    return scaled_H, target_dim

# Resize image function
def hconcat_resize_min(im_list, interpolation=cv2.INTER_CUBIC):
    h_min = min(im.shape[0] for im in im_list)
    im_list_resize = [cv2.resize(im, (int(im.shape[1] * h_min / im.shape[0]), h_min), interpolation=interpolation)
                      for im in im_list]
    return cv2.hconcat(im_list_resize)

# Resize image function
def vconcat_resize_min(im_list, interpolation=cv2.INTER_CUBIC):
    w_min = min(im.shape[1] for im in im_list)
    im_list_resize = [cv2.resize(im, (w_min, int(im.shape[0] * w_min / im.shape[1])), interpolation=interpolation)
                      for im in im_list]
    return cv2.vconcat(im_list_resize)

In [ ]:
from math import radians, pi
import numpy as np

# Test with different parameters
# fx = 2349.595
# fy = 1662.2         #1700.688
# roll_from_horizon = 0 #-0.4013
# my_tilt = 0.628319 #0.43695
# orig_width = 1920
# orig_height = 1080

# Obtain overhead birds-eye-view projection.
overhead_hmatrix, est_range_u, est_range_v = get_overhead_hmatrix_from_4cameraparams(fx=results['f_pix'], fy=results['f_pix'],
                                                                                         my_tilt=results['vfov'],
                                                                                         my_roll=-radians(
                                                                                             results['roll']),
                                                                                         img_dims=(orig_img_w,
                                                                                                   orig_img_h),
                                                                                         verbose=True)

scaled_overhead_hmatrix, target_dim = get_scaled_homography(overhead_hmatrix, orig_img_h*2, est_range_u, est_range_v)
print(f'Target Dimensions: {target_dim}')
output_dim = (orig_img_w, orig_img_h)
print(f'scaled overhead hmatrix:\n{scaled_overhead_hmatrix}')

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt

# 4 point pairs - p1, p2, p3, p4
points_birdseye = [[400,695],[828,770],[325,1181],[716,1263]]
points_image = [[634,411],[1316,469],[231,717],[1031,847]]

# Get homography matrix through manual annotation
h_mat, status = cv2.findHomography(np.array(points_image), np.array(points_birdseye))

# Read in image screenshot
# img_path = 'homography/OxfordTownCenter.png'
img_path = os.path.join(screenshot_dir,filename)
print(img_path)
img_cv = cv2.imread(img_path)
img_cv = cv2.cvtColor(img_cv, cv2.COLOR_BGR2RGB)

# Perform perspective warping
manual_dim = (1080,1465)
overhead_dim = (2662,2160)
overhead_dim2 = (2987,1440)
overhead_dim3 = (1508,2160)

print(f'Homography Matrix: {h_mat}')
print(f'Auto Homography Matrix: {scaled_overhead_hmatrix}')

warped_image = cv2.warpPerspective(img_cv, scaled_overhead_hmatrix, dsize=target_dim, flags=cv2.INTER_CUBIC)
plt.figure(figsize=[10,10])
plt.imshow(warped_image)
plt.show()


## Run Social Force Model plots
Only for TownCenter_Short video for now<br>
Pipeline is not fully real-time yet as social force model requires computation of BOW across entire file stream

In [ ]:
import numpy as np

if bool_towncenter is True:

    # Load Force Flows
	force_flow_path = 'TownCentre_flow/video_flowvector_TownCentre_0.3.npy'
	force_flow = np.load(force_flow_path)

	# Load likelihoods
	xdata = np.load('TownCentre_flow/TownCentre_liks_xdata.npy')
	ydata = np.load('TownCentre_flow/TownCentre_liks_ydata.npy')

if bool_applestore is True:

    # Load Force Flows
	force_flow_path = 'AppleStore_flow/video_flowvector_AppleStoreShooting_01_0.5.npy'
	force_flow = np.load(force_flow_path)

	# Load likelihoods
	xdata = np.load('AppleStore_flow/AppleStoreShooting_01_liks_xdata.npy')
	ydata = np.load('AppleStore_flow/AppleStoreShooting_01_liks_ydata.npy')

# Run Analytics Pipeline
- Detector and Tracker
- Homography transformation to birds eye view
- Social Distancing violation based on BEV projection
- Crowd Violence Inference
- Anomalies log-likelihood analysis through LDA

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

import cv2
import pyshine as ps
import scipy
import numpy as np
from collections import deque
import itertools
import math

# Test LOF algorithm
from sklearn.neighbors import LocalOutlierFactor

COLOR_RED = (0, 0, 255)
COLOR_GREEN = (0, 255, 0)
COLOR_BLUE = (255, 0, 0)

distance_minimum = 50

# Use Auto h_mat
# height = 2160
# width = 2662
height = target_dim[1]
width = target_dim[0]
scale_resize = 40

# Create a deque buffer to store 4 secs worth of frames (120 frames for 30fps) to input into c3d
buffer_len = 120
c3d_videoInput = deque(maxlen=buffer_len)

# buffering parameters to reduce output instability
consec_action = 0
prev_action = 0

# Initialise action output
final_action = 0
# Initialise SMM Violation
smm_volation = 0
# Initialise Anomaly Detection
anomaly_detected = False
liks_thres = -40000

fig = plt.figure(figsize=(10,10), dpi=300)
plt.grid()
plt.ylabel('Scores')
plt.xlabel('Frame_idx')
plt.title('LDA Log-Likelihood Scores Across Frames')
logo = plt.imread('icarus_logo.jpg')
fig.figimage(logo, 1100, 1000, alpha=.30, zorder=1)
line, = plt.plot(xdata, ydata, color='red')
xdata_fig = list()
ydata_fig = list()

with torch.no_grad():
    for frame_idx, (path, img, im0s, vid_cap, s) in enumerate(dataset):
        
        print(f'FRAME_INDEX: {frame_idx}')
        # Copy img frame for c3d processing
        img_c3d = img.copy()

        # Initialise Violation count back to 0 every new frame
        smm_volation = 0

        # collect 120 frames for c3d
        # if frame_idx <= buffer_len - 1:
        if (frame_idx+1) % buffer_len != 0:
            img_c3d = img_c3d.transpose((1,2,0))
            # resize image frame and convert to grayscale
            img_c3d = cv2.resize(img_c3d, (c3d_height, c3d_width))
            c3d_videoInput.append(cv2.cvtColor(img_c3d, cv2.COLOR_BGR2GRAY))
        else:
            startc3d_time = time.time()
            # Convert video segment to 3D input
            curr_action = datato3D.inference(np.array(c3d_videoInput))
            print(f'c3d_compute_time: {time.time() - startc3d_time} secs')
            print(f'Immediate_output: {curr_action}')

            # 5 consecutive agreeing output before we decide that VD has occurred
            # This is to remove frame-by-frame prediction flickering instability
            if curr_action == prev_action:
                consec_action += 1
                action = curr_action
            else:
                # Reset counter here
                consec_action = 0

            if consec_action == 3:
                final_action = action
                # Reset counter here
                consec_action = 0
                
            # Assign to prev state
            prev_action = curr_action
            img_c3d = img_c3d.transpose((1,2,0))
            # resize image frame and convert to grayscale
            img_c3d = cv2.resize(img_c3d, (c3d_height, c3d_width))
            c3d_videoInput.append(cv2.cvtColor(img_c3d, cv2.COLOR_BGR2GRAY))

        # Create a blank map canvas every frame
        map_image = np.zeros((height,width,3), np.uint8)
        map_pt_array = []
        bbox_pt_array = []   

        t1 = time_sync()
        img = torch.from_numpy(img).to(device)
        img = img.half() if half else img.float()  # uint8 to fp16/32
        img /= 255.0  # 0 - 255 to 0.0 - 1.0
        if img.ndimension() == 3:
            img = img.unsqueeze(0)
        t2 = time_sync()
        dt[0] += t2 - t1

        # Inference
        visualize = increment_path(save_dir / Path(path).stem, mkdir=True) if visual else False
        pred = model_yolo(img, augment=augment, visualize=visualize)
        t3 = time_sync()
        dt[1] += t3 - t2

        # Apply NMS
        pred = non_max_suppression(pred, conf_thres, iou_thres, classes, agnostic_nms, max_det=max_det)
        dt[2] += time_sync() - t3

        # Process detections
        for i, det in enumerate(pred):  # detections per image
            seen += 1
            p, im0, _ = path, im0s.copy(), getattr(dataset, 'frame', 0)

            p = Path(p)  # to Path
            save_path = str(save_dir / p.name)  # im.jpg, vid.mp4, ...
            s += '%gx%g ' % img.shape[2:]  # print string

            # annotator = Annotator(im0, line_width=2, pil=not ascii)      

            if det is not None and len(det):
                # Rescale boxes from img_size to im0 size
                det[:, :4] = scale_coords(
                    img.shape[2:], det[:, :4], im0.shape).round()

                # Print results
                for c in det[:, -1].unique():
                    n = (det[:, -1] == c).sum()  # detections per class
                    s += f"{n} {names[int(c)]}{'s' * (n > 1)}, "  # add to string

                xywhs = xyxy2xywh(det[:, 0:4])
                confs = det[:, 4]
                clss = det[:, 5]

                # pass detections to deepsort
                t4 = time_sync()
                outputs = deepsort.update(xywhs.cpu(), confs.cpu(), clss.cpu(), im0)
                num_persons = len(outputs)
                t5 = time_sync()
                dt[3] += t5 - t4

                # draw boxes for visualization
                if len(outputs) > 0:
                    for j, (output, conf) in enumerate(zip(outputs, confs)):

                        bboxes = output[0:4]
                        id = output[4]
                        cls = output[5]

                        # Box definitions
                        x = int(output[0])
                        y = int(output[1])
                        w = int(output[2] - output[0])
                        h = int(output[3] - output[1])
                        bbox_int = x,y,w,h

                        c = int(cls)  # integer class
                        label = f'ID:{id} ({conf:.2f})'
                        # print(im0.shape)
                        # Write Track ID on target base
                        cv2.putText(im0,str(id),(x,y+h+20), 0, 400*1e-3*2, [255,255,255], 2)

                        # Comment out this portion and draw our own boxes
                        # annotator.box_label(bboxes, label, color=colors(c, True))

                        # Compute box base - Homography transformation
                        bbox_pt = np.array([(int(output[0] + (output[2] - output[0])/2), int(output[1]))], dtype='float32')
                        bbox_pt = np.array([bbox_pt], dtype='float32')
                        bbox_pt_array.append((int(output[0]), int(output[1]), int(output[2]), int(output[3])))

                        # Obtain corresponding birdseye point transformation
                        # - manual h_mat
                        # - auto h_mat (overhead_hmat)
                        # birdseye_pt = cv2.perspectiveTransform(bbox_pt, h_mat)
                        birdseye_pt = cv2.perspectiveTransform(bbox_pt, scaled_overhead_hmatrix)                        
                        map_pt = (int(birdseye_pt[0][0][0]), int(birdseye_pt[0][0][1]))
                        map_pt_array.append(map_pt)

                        # Draw point on 2d plane
                        cv2.circle(map_image, map_pt, 20, COLOR_GREEN, -2)
                        # box_pt = (int(bbox_pt[0][0][0]), int(bbox_pt[0][0][1]))
                        # cv2.circle(map_image, box_pt, 20, (0,255,0), -2)
                        

                        if save_txt:
                            # to MOT format
                            bbox_left = output[0]
                            bbox_top = output[1]
                            bbox_w = output[2] - output[0]
                            bbox_h = output[3] - output[1]
                            # Write MOT compliant results to file
                            with open(txt_path, 'a') as f:
                                f.write(('%g ' * 10 + '\n') % (frame_idx + 1, id, bbox_left,  # MOT format
                                                                bbox_top, bbox_w, bbox_h, -1, -1, -1, -1))

                LOGGER.info(f'{s}Done. YOLO:({t3 - t2:.3f}s), DeepSort:({t5 - t4:.3f}s)')

                #************** SMM Violations based on BEV Projections **************
                # Iterate over every possible 2 by 2 between the points combinations 
                list_indexes = list(itertools.combinations(range(len(map_pt_array)), 2))
                for i,pair in enumerate(itertools.combinations(map_pt_array, r=2)):

                    # Check if the distance between each combination of points is less than the minimum distance chosen
                    if math.sqrt( (pair[0][0] - pair[1][0])**2 + (pair[0][1] - pair[1][1])**2 ) < int(distance_minimum):
                        
                        # Change the colors of the points that are too close from each other to red
                        if not (pair[0][0] > width or pair[0][0] < 0 or pair[0][1] > height+200  or pair[0][1] < 0 or pair[1][0] > width or pair[1][0] < 0 or pair[1][1] > height+200  or pair[1][1] < 0):
                            
                            # For every violation pair, we add a count to the SMM violation
                            smm_volation += 1

                            # Change colours to RED
                            cv2.circle(map_image, (pair[0][0],pair[0][1]), 60, COLOR_RED, 2)
                            cv2.circle(map_image, (pair[0][0],pair[0][1]), 20, COLOR_RED, -2)
                            cv2.circle(map_image, (pair[1][0],pair[1][1]), 60, COLOR_RED, 2)
                            cv2.circle(map_image, (pair[1][0],pair[1][1]), 20, COLOR_RED, -2)

                            # Get the equivalent indexes of these points in the original frame and change the color to red
                            index_pt1 = list_indexes[i][0]
                            index_pt2 = list_indexes[i][1]
                            
                            # Highlight violations in red rectangle boxes
                            cv2.rectangle(im0, (bbox_pt_array[index_pt1][0],bbox_pt_array[index_pt1][1]),(bbox_pt_array[index_pt1][2],bbox_pt_array[index_pt1][3]), COLOR_RED, 2)
                            cv2.rectangle(im0, (bbox_pt_array[index_pt2][0],bbox_pt_array[index_pt2][1]),(bbox_pt_array[index_pt2][2],bbox_pt_array[index_pt2][3]), COLOR_RED, 2)

                #************** LOCAL OUTLIER FACTOR **************
                # For testing only

                # Fit model for LOF detection
                # if len(map_pt_array) != 0:
                #     clf = LocalOutlierFactor(n_neighbors=4, metric='euclidean', contamination=0.1)
                #     y_pred = clf.fit_predict(map_pt_array)
                #     x_scores = clf.negative_outlier_factor_
                #     for i in range(len(x_scores)):
                #         if x_scores[i] > -1.0:
                #             cv2.circle(map_image, map_pt_array[i], 20, (0,0,255), -2)
                    
                    # Plot circles with radius proportional to the outlier score
                    # radius = (x_scores.max() - x_scores) / (x_scores.max() - x_scores.min())
                    # radius = (x_scores - x_scores.min()) / (x_scores.max() - x_scores.min())
                    # print(f'radius: {radius}')
                    # for i in range(len(radius)):
                    #     cv2.circle(map_image, map_pt_array[i], int(100*radius[i]), (255,255,255), 5)
            else:
                deepsort.increment_ages()
                LOGGER.info('No detections')
            
            # Display Output for Crowd Size
            text = f'Crowd Size: {num_persons}'
            image = ps.putBText(im0,text,text_offset_x=im0.shape[1]-420,text_offset_y=im0.shape[0]-170,\
                                vspace=10,hspace=10, font_scale=1.0,background_RGB=(255,255,255),\
                                text_RGB=(0,0,0))
            
            # Display output for Social Distancing
            text = f'SMM Violations: {smm_volation}'
            image = ps.putBText(image,text,text_offset_x=im0.shape[1]-420,text_offset_y=im0.shape[0]-128,\
                                    vspace=10,hspace=10, font_scale=1.0,background_RGB=(255,255,255),\
                                    text_RGB=(0,0,0))

            if bool_towncenter is True or bool_applestore is True:
                if ydata[frame_idx] <= liks_thres:
                    anomaly_detected = True
                else:
                    anomaly_detected = False

            if anomaly_detected == True:
                # Display output for Anomaly Detection
                text = f'Crowd Anomaly Detected'
                image = ps.putBText(image,text,text_offset_x=im0.shape[1]-420,text_offset_y=im0.shape[0]-86,\
                                        vspace=10,hspace=10,font_scale=1.0,background_RGB=(255,255,255),\
                                        text_RGB=(255,0,0))
            else:
                text = f'No Anomaly Detected'
                image = ps.putBText(image,text,text_offset_x=im0.shape[1]-420,text_offset_y=im0.shape[0]-86,\
                                        vspace=10,hspace=10,font_scale=1.0,background_RGB=(255,255,255),\
                                        text_RGB=(0,0,0))                

            # Display output for Violence Detection
            if final_action == 1:
                text  =  'Violence Detected'
                image = ps.putBText(image,text,text_offset_x=im0.shape[1]-420,text_offset_y=im0.shape[0]-44,\
                                    vspace=10,hspace=10,font_scale=1.0,background_RGB=(255,255,255),\
                                    text_RGB=(255,0,0))
            else:
                text  =  'No Violence Detected'
                image = ps.putBText(image,text,text_offset_x=im0.shape[1]-420,text_offset_y=im0.shape[0]-44,\
                                    vspace=10,hspace=10,font_scale=1.0,background_RGB=(255,255,255),\
                                    text_RGB=(0,0,0))
            
            # Plot results by joining the images together
            # scale and resize the image smaller for display
            # image = im0.copy()  # For testing only - to remove
            final_image_joined = hconcat_resize_min([image, map_image])
            top_resize_w = int(final_image_joined.shape[1] * scale_resize/100)
            top_resize_h = int(final_image_joined.shape[0] * scale_resize/100)
            top_resize_dim = (top_resize_w, top_resize_h)
            top_resized_image = cv2.resize(final_image_joined, top_resize_dim, interpolation=cv2.INTER_AREA)

            if bool_towncenter is True or bool_applestore is True:
                #**********************************************************
                # Plot and output force flow anomaly detection here
                #**********************************************************
                # 1st: need to resize force flow vector to be same as img
                force_flow_resize = cv2.resize(force_flow[frame_idx], (im0s.shape[1],im0s.shape[0]))
                
                # 2nd: Create graph to plot likelihoods
                xdata_fig.append(frame_idx)
                ydata_fig.append(ydata[frame_idx])
                line.set_data(xdata_fig, ydata_fig)

                # redraw the canvas
                fig.canvas.draw()
                # # convert canvas to image
                graphimg = np.fromstring(fig.canvas.tostring_rgb(), dtype=np.uint8,
                        sep='')
                graphimg  = graphimg.reshape(fig.canvas.get_width_height()[::-1] + (3,))
                # img is rgb, convert to opencv's default bgr
                graphimg = cv2.cvtColor(graphimg,cv2.COLOR_RGB2BGR)
                graphimg = cv2.resize(graphimg, (width, height))

                force_image_joined = hconcat_resize_min([force_flow_resize, graphimg])

                bottom_resize_w = int(force_image_joined.shape[1] * scale_resize/100)
                bottom_resize_h = int(force_image_joined.shape[0] * scale_resize/100)
                bottom_resize_dim = (bottom_resize_w, bottom_resize_h)
                bottom_resized_image = cv2.resize(force_image_joined, bottom_resize_dim, interpolation=cv2.INTER_AREA)

                # Concat top and bottom together
                final_display = vconcat_resize_min([top_resized_image, bottom_resized_image])
            else:
                final_display = top_resized_image

            cv2.imshow(str(p), final_display)
            if cv2.waitKey(1) == ord('q'):  # q to quit
                raise StopIteration

            # # Stream results
            # im0 = annotator.result()
            # if show_vid:
            #     cv2.imshow(str(p), im0)
            #     if cv2.waitKey(1) == ord('q'):  # q to quit
            #         raise StopIteration
        
        # img_out = sd.proximity_evaluation(trackid_bbox_centroid, im0, cam)

        # # Plot results
        # cv2.imshow(str(p), output_img)
        # if cv2.waitKey(1) == ord('q'):  # q to quit
        #     raise StopIteration

            # Save results (image with detections)
            if save_vid:
                if vid_path != save_path:  # new video
                    vid_path = save_path
                    if isinstance(vid_writer, cv2.VideoWriter):
                        vid_writer.release()  # release previous video writer
                    # if vid_cap:  # video
                    #     fps = vid_cap.get(cv2.CAP_PROP_FPS)
                    #     w = int(vid_cap.get(cv2.CAP_PROP_FRAME_WIDTH))
                    #     h = int(vid_cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
                    # else:  # stream
                    fps, w, h = 30, final_display.shape[1], final_display.shape[0]

                    # fourcc = cv2.VideoWriter_fourcc('m', 'p', '4', 'v')
                    vid_writer = cv2.VideoWriter(save_path, cv2.VideoWriter_fourcc(*'mp4v'), fps, (w, h), True)
                vid_writer.write(final_display)

        if bool_towncenter is True or bool_applestore is True:
            if frame_idx == len(xdata)-1:
                break
    
    vid_cap.release()
    vid_writer.release()
    cv2.destroyAllWindows()